In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import col
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.mllib.evaluation import RankingMetrics
import plotly.express as px
import random 
import time
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

# for NDCG
from pyspark.sql import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import expr
import pyspark.sql.functions as F

# for lightFM
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

import os
import zipfile
import requests

import numpy as np

/Users/jessie/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning:

LightFM was compiled without OpenMP support. Only a single thread will be used.



In [66]:
line_count = len(open("/Users/jessie/Desktop/Personalization/Project2/yelp_dataset/user.json").readlines())
user_ids, yelping_sinces = [], []
# usefuls, funnys, cools = [], [], []
fanss, elites = [], []
compliment_hots, compliment_mores, compliment_profiles, compliment_cutes = [], [], [], []
compliment_lists, compliment_notes, compliment_plains, compliment_cools = [], [], [], []
compliment_funnys, compliment_writers, compliment_photos = [], [], []
with open("/Users/jessie/Desktop/Personalization/Project2/yelp_dataset/user.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        #review_counts += [blob["review_count"]]
        yelping_sinces += [blob["yelping_since"]]
        fanss += [blob["fans"]] 
        elites += [blob["elite"]]
        compliment_hots += [blob["compliment_hot"]]
        compliment_mores += [blob["compliment_more"]]
        compliment_profiles += [blob["compliment_profile"]]
        compliment_cutes += [blob["compliment_cute"]]
        compliment_lists += [blob["compliment_list"]]
        compliment_notes += [blob["compliment_note"]]
        compliment_plains += [blob["compliment_plain"]]
        compliment_cools += [blob["compliment_cool"]]
        compliment_funnys += [blob["compliment_funny"]]
        compliment_writers += [blob["compliment_writer"]]
        compliment_photos += [blob["compliment_photos"]]

users = pd.DataFrame(
{"user_id": user_ids, "yelping_since": yelping_sinces, "fans": fanss, "elite": elites, 
"compliment_hot": compliment_hots, "compliment_more":compliment_mores, "compliment_profile": compliment_profiles,
"compliment_cute":compliment_cutes, "compliment_list":compliment_lists,
"compliment_note": compliment_notes, "compliment_plain": compliment_plains,
"compliment_cool": compliment_cools, "compliment_funny": compliment_funnys,
"compliment_writer": compliment_writers, "compliment_photos": compliment_photos}
)

100%|██████████| 1637138/1637138 [00:33<00:00, 49015.06it/s]


In [67]:
# save and load the dataframe
#users.to_pickle('users.pkl')

In [2]:
users = pd.read_pickle('users.pkl')
ratings = pd.read_pickle('ratings.pkl')
business = pd.read_pickle('business.pkl')

In [3]:
ratings.head()

,user_id,business_id,rating,date
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36
1,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33
2,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03
3,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38
4,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38


In [4]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [68]:
users.head()

,user_id,yelping_since,fans,elite,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,2013-10-08 23:11:33,5,"2015,2016,2017",2,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,2013-02-21 22:29:06,4,,1,0,0,0,0,0,0,1,1,0,0
2,bc8C_eETBWL0olvFSJJd0w,2013-10-04 00:16:10,0,,0,0,0,0,0,1,0,0,0,0,0
3,dD0gZpBctWGdWo9WlGuhlA,2014-05-22 15:57:30,5,,1,0,0,0,0,0,2,0,0,1,0
4,MM4RJAeH6yuaN8oZDSt0RA,2013-10-23 07:02:50,39,"2015,2016,2017,2018",28,1,0,0,1,16,57,80,80,25,5


In [5]:
# select active ratings
user_counts = ratings["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()

Get sub sample  
1% of the user as subsample, the most recent record as subsample test

In [6]:
# randomly select 1% users
import random 
random.seed(12345)
sample_1pc = random.sample(active_users,int(0.01*len(active_users)))
sample_ratings=ratings.loc[ratings['user_id'].isin(sample_1pc)]
sample_business=business.loc[business['business_id'].isin(sample_ratings['business_id'])]
print("number of users in the sample: ",len(sample_1pc))
print("number of business in the sample: ",len(sample_business))
print("number of ratings in the sample: ",len(sample_ratings))

number of users in the sample:  2861
number of business in the sample:  28191
number of ratings in the sample:  49133


In [7]:
# reset the index 
sample_ratings = sample_ratings.reset_index(drop=True)
sample_business = sample_business.reset_index(drop=True)

# Feature selection 1: stars + states + review_counts + categories

In [20]:
# prepare the dataset
dataset = Dataset()
dataset.fit((sample_ratings['user_id']),
            (sample_ratings['business_id']))
dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['stars']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['state']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['review_count']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['categories']))


(interactions, weights) = dataset.build_interactions((sample_ratings['user_id'][i],sample_ratings['business_id'][i]) 
                                                     for i in range(len(sample_ratings)))

print(repr(interactions))

item_features = dataset.build_item_features(((sample_business['business_id'][i], [sample_business['stars'][i],
                             sample_business['state'][i],sample_business['review_count'][i],sample_business['categories'][i]])
                                              for i in range(len(sample_business))))
print(repr(item_features))


<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 49133 stored elements in COOrdinate format>
<28191x48536 sparse matrix of type '<class 'numpy.float32'>'
	with 140719 stored elements in Compressed Sparse Row format>


In [21]:
#from lightfm import cross_validation
from lightfm.cross_validation import random_train_test_split

In [23]:
my_r=np.random.seed(12345)
training, test = random_train_test_split(interactions, test_percentage=0.2, random_state=my_r)

In [24]:
training

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 39306 stored elements in COOrdinate format>

In [25]:
test

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 9827 stored elements in COOrdinate format>

In [26]:
# FM
model = LightFM(loss='warp')
model.fit(training,epochs=20,item_features=item_features)

In [27]:
train_precision = precision_at_k(model, training,item_features=item_features, k=10).mean()
train_auc = auc_score(model, training,item_features=item_features).mean()
print("training precision: ",train_precision)
print("training AUC: ",train_auc)

training precision:  0.031247815
training AUC:  0.9411819


In [28]:
test_precision = precision_at_k(model, test,item_features=item_features, k=10).mean()
test_auc = auc_score(model, test,item_features=item_features).mean()
print("test precision: ",test_precision)
print("test AUC: ",test_auc)

test precision:  0.0051112943
test AUC:  0.88237303


# Feature selection 2: stars + states + review_counts

In [30]:
# prepare the dataset
dataset = Dataset()
dataset.fit((sample_ratings['user_id']),
            (sample_ratings['business_id']))
dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['stars']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['state']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['review_count']))



(interactions, weights) = dataset.build_interactions((sample_ratings['user_id'][i],sample_ratings['business_id'][i]) 
                                                     for i in range(len(sample_ratings)))

print(repr(interactions))

item_features = dataset.build_item_features(((sample_business['business_id'][i], [sample_business['stars'][i],
                             sample_business['state'][i],sample_business['review_count'][i]])
                                              for i in range(len(sample_business))))

print(repr(item_features))

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 49133 stored elements in COOrdinate format>
<28191x29394 sparse matrix of type '<class 'numpy.float32'>'
	with 112528 stored elements in Compressed Sparse Row format>


In [31]:
#from lightfm import cross_validation
from lightfm.cross_validation import random_train_test_split

In [32]:
my_r=np.random.seed(12345)
training, test = random_train_test_split(interactions, test_percentage=0.2, random_state=my_r)

In [33]:
training

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 39306 stored elements in COOrdinate format>

In [34]:
test

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 9827 stored elements in COOrdinate format>

In [35]:
# FM
model = LightFM(loss='warp')
model.fit(training,epochs=20,item_features=item_features)

In [36]:
train_precision = precision_at_k(model, training,item_features=item_features, k=10).mean()
train_auc = auc_score(model, training,item_features=item_features).mean()
print("training precision: ",train_precision)
print("training AUC: ",train_auc)

training precision:  0.027437959
training AUC:  0.93710303


In [37]:
test_precision = precision_at_k(model, test,item_features=item_features, k=10).mean()
test_auc = auc_score(model, test,item_features=item_features).mean()
print("test precision: ",test_precision)
print("test AUC: ",test_auc)

test precision:  0.0061512887
test AUC:  0.8836798


# Feature selection 3: stars + states

In [38]:
# prepare the dataset
dataset = Dataset()
dataset.fit((sample_ratings['user_id']),
            (sample_ratings['business_id']))
dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['stars']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['state']))



(interactions, weights) = dataset.build_interactions((sample_ratings['user_id'][i],sample_ratings['business_id'][i]) 
                                                     for i in range(len(sample_ratings)))

print(repr(interactions))

item_features = dataset.build_item_features(((sample_business['business_id'][i], [sample_business['stars'][i],
                             sample_business['state'][i]])
                                              for i in range(len(sample_business))))

print(repr(item_features))

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 49133 stored elements in COOrdinate format>
<28191x28213 sparse matrix of type '<class 'numpy.float32'>'
	with 84573 stored elements in Compressed Sparse Row format>


In [39]:
#from lightfm import cross_validation
from lightfm.cross_validation import random_train_test_split

In [40]:
my_r=np.random.seed(12345)
training, test = random_train_test_split(interactions, test_percentage=0.2, random_state=my_r)

In [41]:
training

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 39306 stored elements in COOrdinate format>

In [42]:
test

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 9827 stored elements in COOrdinate format>

In [43]:
# FM
model = LightFM(loss='warp')
model.fit(training,epochs=20,item_features=item_features)

In [44]:
train_precision = precision_at_k(model, training,item_features=item_features, k=10).mean()
train_auc = auc_score(model, training,item_features=item_features).mean()
print("training precision: ",train_precision)
print("training AUC: ",train_auc)

training precision:  0.03293706
training AUC:  0.94871986


In [45]:
test_precision = precision_at_k(model, test,item_features=item_features, k=10).mean()
test_auc = auc_score(model, test,item_features=item_features).mean()
print("test precision: ",test_precision)
print("test AUC: ",test_auc)

test precision:  0.0050000004
test AUC:  0.8778921


# Feature selection 4: stars + review_counts

In [46]:
# prepare the dataset
dataset = Dataset()
dataset.fit((sample_ratings['user_id']),
            (sample_ratings['business_id']))
dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['stars']))


dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['review_count']))



(interactions, weights) = dataset.build_interactions((sample_ratings['user_id'][i],sample_ratings['business_id'][i]) 
                                                     for i in range(len(sample_ratings)))

print(repr(interactions))

item_features = dataset.build_item_features(((sample_business['business_id'][i], [sample_business['stars'][i],
                             sample_business['review_count'][i]])
                                              for i in range(len(sample_business))))

print(repr(item_features))

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 49133 stored elements in COOrdinate format>
<28191x29381 sparse matrix of type '<class 'numpy.float32'>'
	with 84337 stored elements in Compressed Sparse Row format>


In [47]:
#from lightfm import cross_validation
from lightfm.cross_validation import random_train_test_split

In [48]:
my_r=np.random.seed(12345)
training, test = random_train_test_split(interactions, test_percentage=0.2, random_state=my_r)

In [49]:
training

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 39306 stored elements in COOrdinate format>

In [50]:
test

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 9827 stored elements in COOrdinate format>

In [51]:
# FM
model = LightFM(loss='warp')
model.fit(training,epochs=20,item_features=item_features)

In [52]:
train_precision = precision_at_k(model, training,item_features=item_features, k=10).mean()
train_auc = auc_score(model, training,item_features=item_features).mean()
print("training precision: ",train_precision)
print("training AUC: ",train_auc)

training precision:  0.017056974
training AUC:  0.8643199


In [53]:
test_precision = precision_at_k(model, test,item_features=item_features, k=10).mean()
test_auc = auc_score(model, test,item_features=item_features).mean()
print("test precision: ",test_precision)
print("test AUC: ",test_auc)

test precision:  0.002324616
test AUC:  0.62150717


# Feature selection 5: states + review_counts

In [54]:
# prepare the dataset
dataset = Dataset()
dataset.fit((sample_ratings['user_id']),
            (sample_ratings['business_id']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['state']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['review_count']))



(interactions, weights) = dataset.build_interactions((sample_ratings['user_id'][i],sample_ratings['business_id'][i]) 
                                                     for i in range(len(sample_ratings)))

print(repr(interactions))

item_features = dataset.build_item_features(((sample_business['business_id'][i], [
                             sample_business['state'][i],sample_business['review_count'][i]])
                                              for i in range(len(sample_business))))

print(repr(item_features))

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 49133 stored elements in COOrdinate format>
<28191x29388 sparse matrix of type '<class 'numpy.float32'>'
	with 84573 stored elements in Compressed Sparse Row format>


In [56]:
#from lightfm import cross_validation
from lightfm.cross_validation import random_train_test_split

In [57]:
my_r=np.random.seed(12345)
training, test = random_train_test_split(interactions, test_percentage=0.2, random_state=my_r)

In [58]:
training

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 39306 stored elements in COOrdinate format>

In [59]:
test

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 9827 stored elements in COOrdinate format>

In [60]:
# FM
model = LightFM(loss='warp')
model.fit(training,epochs=20,item_features=item_features)

In [61]:
train_precision = precision_at_k(model, training,item_features=item_features, k=10).mean()
train_auc = auc_score(model, training,item_features=item_features).mean()
print("training precision: ",train_precision)
print("training AUC: ",train_auc)

training precision:  0.029185599
training AUC:  0.93837315


In [62]:
test_precision = precision_at_k(model, test,item_features=item_features, k=10).mean()
test_auc = auc_score(model, test,item_features=item_features).mean()
print("test precision: ",test_precision)
print("test AUC: ",test_auc)

test precision:  0.0050371597
test AUC:  0.87833965


So the best feature combination is Feature selection 2: stars + states + review_counts, with precision and AUC as follows:  
test precision:  0.0061512887
test AUC:  0.8836798

# Feature selection 6: stars + states + review_counts + year(users) 

In [70]:
users['year']=2019-pd.DatetimeIndex(users['yelping_since']).year

In [73]:
sample_users=users.loc[users['user_id'].isin(sample_ratings['user_id'])]
print("number of users in the sample: ",len(sample_users))

number of users in the sample:  2861


In [75]:
# reset index
sample_users = sample_users.reset_index(drop=True)

In [76]:
# prepare the dataset
dataset = Dataset()
dataset.fit((sample_ratings['user_id']),
            (sample_ratings['business_id']))
dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['stars']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['state']))

dataset.fit_partial(items=(sample_business['business_id']),
                    item_features = (sample_business['review_count']))

dataset.fit_partial(users=(sample_users['user_id']),
                    user_features = (sample_users['year']))


(interactions, weights) = dataset.build_interactions((sample_ratings['user_id'][i],sample_ratings['business_id'][i]) 
                                                     for i in range(len(sample_ratings)))

print(repr(interactions))

item_features = dataset.build_item_features(((sample_business['business_id'][i], [sample_business['stars'][i],
                             sample_business['state'][i],sample_business['review_count'][i]])
                                              for i in range(len(sample_business))))

user_features = dataset.build_user_features(((sample_users['user_id'][i], [sample_users['year'][i]])
                                              for i in range(len(sample_users))))

print(repr(item_features))
print(repr(user_features))

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 49133 stored elements in COOrdinate format>
<28191x29394 sparse matrix of type '<class 'numpy.float32'>'
	with 112528 stored elements in Compressed Sparse Row format>
<2861x2875 sparse matrix of type '<class 'numpy.float32'>'
	with 5722 stored elements in Compressed Sparse Row format>


In [77]:
#from lightfm import cross_validation
from lightfm.cross_validation import random_train_test_split

In [78]:
my_r=np.random.seed(12345)
training, test = random_train_test_split(interactions, test_percentage=0.2, random_state=my_r)

In [79]:
training

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 39306 stored elements in COOrdinate format>

In [80]:
test

<2861x28191 sparse matrix of type '<class 'numpy.int32'>'
	with 9827 stored elements in COOrdinate format>

In [82]:
# FM
model = LightFM(loss='warp')
model.fit(training,epochs=20,item_features=item_features,user_features=user_features)

In [83]:
train_precision = precision_at_k(model, training,item_features=item_features,user_features=user_features, k=10).mean()
train_auc = auc_score(model, training,item_features=item_features,user_features=user_features).mean()
print("training precision: ",train_precision)
print("training AUC: ",train_auc)

training precision:  0.028346732
training AUC:  0.9282547


In [84]:
test_precision = precision_at_k(model, test,item_features=item_features, user_features=user_features,k=10).mean()
test_auc = auc_score(model, test,item_features=item_features,user_features=user_features).mean()
print("test precision: ",test_precision)
print("test AUC: ",test_auc)

test precision:  0.005154639
test AUC:  0.8852935
